# Day 16

## Part 1

We want to apply the FFT algorithm (Flawed Frequency Transmission) to a given signal.

It works by phases.
In each phase you apply some comuutation to the signal, then the new signal becomes the input for the next phase.

In a phase, given the signal `s` which is an ordered list of int, the computation is:
```
s[n] = abs[sum(s[i] * p[i] for i in range(len(s))) % 10
```

`p` is a pattern evolving like this:
* when calculating `s[0]`, the pattern is `[1, 0, -1]`
* when calcuting `s[1]`, the pattern is `[0, 1, 1, 0, 0, -1, -1]`
* when calcuting `s[2]`, the pattern is `[0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1]`
* and so on

If the pattern is shorter than the signal, it reapeats.

In [1]:
from itertools import cycle, repeat

def _get_pattern(position):
    pattern = cycle([0, 1, 0, -1])
    
    for v in pattern:
        yield from repeat(v, position + 1)
            
def get_pattern(position):
    iterator = _get_pattern(position)
    # skip first
    next(iterator)
    return iterator

In [2]:
from itertools import islice

list(islice(get_pattern(0), 0, 20))

[1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0]

In [3]:
list(islice(get_pattern(1), 0, 20))

[0, 1, 1, 0, 0, -1, -1, 0, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, 1, 0]

In [4]:
list(islice(get_pattern(4), 0, 20))

[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, 0]

In [5]:
def apply_phase(signal):
    new_signal = []
    for i in range(len(signal)):
        new_signal.append(abs(sum(s*p for s, p in zip(signal, get_pattern(i)))) % 10)
        
    return new_signal

In [6]:
apply_phase([1,2,3,4,5,6,7,8])

[4, 8, 2, 2, 6, 1, 5, 8]

In [7]:
apply_phase([4, 8, 2, 2, 6, 1, 5, 8])

[3, 4, 0, 4, 0, 4, 3, 8]

In [8]:
apply_phase([3, 4, 0, 4, 0, 4, 3, 8])

[0, 3, 4, 1, 5, 5, 1, 8]

In [9]:
apply_phase([0, 3, 4, 1, 5, 5, 1, 8])

[0, 1, 0, 2, 9, 4, 9, 8]

In [10]:
signal = [8,0,8,7,1,2,2,4,5,8,5,9,1,4,5,4,6,6,1,9,0,8,3,2,1,8,6,4,5,5,9,5]
for _ in range(100):
    signal = apply_phase(signal)

signal[:8]

[2, 4, 1, 7, 6, 1, 7, 6]

In [11]:
%%time

signal = list(
    int(i) for i in
    "59756772370948995765943195844952640015210703313486295362653878290009098923609769261473534009395188480864325959786470084762607666312503091505466258796062230652769633818282653497853018108281567627899722548602257463608530331299936274116326038606007040084159138769832784921878333830514041948066594667152593945159170816779820264758715101494739244533095696039336070510975612190417391067896410262310835830006544632083421447385542256916141256383813360662952845638955872442636455511906111157861890394133454959320174572270568292972621253460895625862616228998147301670850340831993043617316938748361984714845874270986989103792418940945322846146634931990046966552"
)
for _ in range(100):
    signal = apply_phase(signal)
    
signal[:8]

CPU times: user 3.13 s, sys: 0 ns, total: 3.13 s
Wall time: 3.13 s


[6, 9, 5, 4, 9, 1, 5, 5]

## Part 2

This time we take our previous signal but duplicates it 10000 times!

We need to read the 8 digits after 100 phases, and after an offset defined by the number constructed with the 7 first digits.

Our previous attempt isn't fast enough, we need something better.
We can notice the pattern actually tells us which number to add (+1) and which to substract (-1).
The pattern grows like this:
* `[1, 0, -1]`: add from 0 to 1 (excluded), substract from 2 to 3 (excluded)
* `[0, 1, 1, 0, 0, -1, -1]`: add from 1 to 3, substract from 5 to 7
* `[0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1]`: add from 2 to 5, substract from 8 to 11

In [20]:
def apply_phase(signal, count):
    new_signal = signal.copy()
    signal_len = len(signal)
    
    for _ in range(count):
        # We can resume the pattern to the position where 1 and -1 start and end.
        # The pattern length is equal to minus_end.
        sum_start = 0
        sum_end = 1
        minus_start = 2
        minus_end = 3
        
        for i in range(signal_len):
            jumps = (signal_len - 1) // minus_end
            value = 0
            for j in range(jumps + 1):
                #print(sum_start + minus_end * j, sum_end + minus_end * j)
                value += sum(new_signal[sum_start + minus_end * j:sum_end + minus_end * j]) - sum(new_signal[minus_start + minus_end * j:minus_end * (j+1)])
                
            new_signal[i] = abs(value) % 10
        
            sum_start += 1
            sum_end += 2
            minus_start += 3
            minus_end += 4
        
    return new_signal

In [21]:
%%time

signal = list(
    int(i) for i in
    "59756772370948995765943195844952640015210703313486295362653878290009098923609769261473534009395188480864325959786470084762607666312503091505466258796062230652769633818282653497853018108281567627899722548602257463608530331299936274116326038606007040084159138769832784921878333830514041948066594667152593945159170816779820264758715101494739244533095696039336070510975612190417391067896410262310835830006544632083421447385542256916141256383813360662952845638955872442636455511906111157861890394133454959320174572270568292972621253460895625862616228998147301670850340831993043617316938748361984714845874270986989103792418940945322846146634931990046966552"
)

signal = apply_phase(signal, 100)
    
signal[:8]

CPU times: user 275 ms, sys: 0 ns, total: 275 ms
Wall time: 273 ms


[9, 1, 3, 6, 2, 5, 1, 8]

That's still not enough.
We can notice that ne pattern for a position `n` has `n-1` zerors at the begining.
We can use that by simpli ignoring the `offset - 1` values.

In [51]:
def apply_phase(signal, count, offset):
    new_signal = signal.copy()
    signal_len = len(signal)
    
    for phase in range(count):
        print(phase)
        # We can resume the pattern to the position where 1 and -1 start and end.
        # The pattern length is equal to minus_end.
        sum_start = 1 * (offset - 1)
        sum_end = 1 + 2 * (offset - 1)
        minus_start = 2 + 3 * (offset - 1)
        minus_end = 3 + 4 * (offset - 1)
        
        for i in range(offset, signal_len):
            jumps = (signal_len - 1) // minus_end
            if i % 1000 == 0:
                print(i, jumps, sum_start, sum_end, minus_start, minus_end)
            value = 0
            
            new_signal[i] = sum(new_signal[i-1:])
            sum_start += 1
            continue
            for j in range(jumps + 1):
                #print(sum_start + minus_end * j, sum_end + minus_end * j)
                value += sum(new_signal[sum_start + minus_end * j:sum_end + minus_end * j]) - sum(new_signal[minus_start + minus_end * j:minus_end * (j+1)])
                
            new_signal[i] = abs(value) % 10
        
            sum_start += 1
            sum_end += 2
            minus_start += 3
            minus_end += 4
            if i % 1000 == 0:
                print(i, jumps, sum_start, sum_end, minus_start, minus_end)
        
    return new_signal

In [79]:
%%time

from functools import lru_cache
from math import ceil

signal = list(
    int(i) for i in
    "59756772370948995765943195844952640015210703313486295362653878290009098923609769261473534009395188480864325959786470084762607666312503091505466258796062230652769633818282653497853018108281567627899722548602257463608530331299936274116326038606007040084159138769832784921878333830514041948066594667152593945159170816779820264758715101494739244533095696039336070510975612190417391067896410262310835830006544632083421447385542256916141256383813360662952845638955872442636455511906111157861890394133454959320174572270568292972621253460895625862616228998147301670850340831993043617316938748361984714845874270986989103792418940945322846146634931990046966552"
)
offset = 5975677

len_signal = len(signal)
sum_signal = sum(signal)

len_total_signal = len_signal * 10000
full_signal = [0 for _ in range(len_signal * 10000)]

@lru_cache(maxsize=None)
def compute_sum(i, phase):
    return sum(signal[i:])

for phase in range(100):
    sum_signal = sum(signal)
    print(phase, signal, sum_signal)
    for i in range(offset, len_total_signal):
        full_signal[i] = abs(compute_sum(i % len_signal, phase) + sum_signal * (len_total_signal - i) // len_signal) % 10
    
    signal = full_signal[ceil(offset / len_signal)*len_signal:ceil(offset / len_signal)*len_signal + len_signal]
    
full_signal[offset:offset + 8]

0 [5, 9, 7, 5, 6, 7, 7, 2, 3, 7, 0, 9, 4, 8, 9, 9, 5, 7, 6, 5, 9, 4, 3, 1, 9, 5, 8, 4, 4, 9, 5, 2, 6, 4, 0, 0, 1, 5, 2, 1, 0, 7, 0, 3, 3, 1, 3, 4, 8, 6, 2, 9, 5, 3, 6, 2, 6, 5, 3, 8, 7, 8, 2, 9, 0, 0, 0, 9, 0, 9, 8, 9, 2, 3, 6, 0, 9, 7, 6, 9, 2, 6, 1, 4, 7, 3, 5, 3, 4, 0, 0, 9, 3, 9, 5, 1, 8, 8, 4, 8, 0, 8, 6, 4, 3, 2, 5, 9, 5, 9, 7, 8, 6, 4, 7, 0, 0, 8, 4, 7, 6, 2, 6, 0, 7, 6, 6, 6, 3, 1, 2, 5, 0, 3, 0, 9, 1, 5, 0, 5, 4, 6, 6, 2, 5, 8, 7, 9, 6, 0, 6, 2, 2, 3, 0, 6, 5, 2, 7, 6, 9, 6, 3, 3, 8, 1, 8, 2, 8, 2, 6, 5, 3, 4, 9, 7, 8, 5, 3, 0, 1, 8, 1, 0, 8, 2, 8, 1, 5, 6, 7, 6, 2, 7, 8, 9, 9, 7, 2, 2, 5, 4, 8, 6, 0, 2, 2, 5, 7, 4, 6, 3, 6, 0, 8, 5, 3, 0, 3, 3, 1, 2, 9, 9, 9, 3, 6, 2, 7, 4, 1, 1, 6, 3, 2, 6, 0, 3, 8, 6, 0, 6, 0, 0, 7, 0, 4, 0, 0, 8, 4, 1, 5, 9, 1, 3, 8, 7, 6, 9, 8, 3, 2, 7, 8, 4, 9, 2, 1, 8, 7, 8, 3, 3, 3, 8, 3, 0, 5, 1, 4, 0, 4, 1, 9, 4, 8, 0, 6, 6, 5, 9, 4, 6, 6, 7, 1, 5, 2, 5, 9, 3, 9, 4, 5, 1, 5, 9, 1, 7, 0, 8, 1, 6, 7, 7, 9, 8, 2, 0, 2, 6, 4, 7, 5, 8, 7, 1, 5, 1, 0, 1, 4

[4, 6, 3, 2, 9, 9, 1, 7]

In [65]:
len(signal)*10000

6500000